In [1]:
import os

os.environ["MLFLOW_TRACKING_URI"] = "https://dagshub.com/ojhamnsh/data_science_mlops.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"] = "ojhamnsh"
os.environ["MLFLOW_TRACKING_PASSWORD"] =  "78ad5f9f4905f1e6a114f217def5f9c446d78ec5"


In [2]:
import os
%pwd

'c:\\Users\\ojha.manish.kumar\\Documents\\Udemy\\DataScience_End_to_End_Projects\\data_science_mlops\\research'

In [3]:
os.chdir("../")
%pwd

'c:\\Users\\ojha.manish.kumar\\Documents\\Udemy\\DataScience_End_to_End_Projects\\data_science_mlops'

In [4]:
%pwd

'c:\\Users\\ojha.manish.kumar\\Documents\\Udemy\\DataScience_End_to_End_Projects\\data_science_mlops'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class ModelEvaluationConfig:
    root_dir: Path
    test_file_path: Path
    model_path: Path
    metric_file_name: Path
    all_params: dict
    target_column: str
    mlflow_uri: str


In [6]:
from src.datascience.constants import *
from src.datascience.utils.common import create_directories, read_yaml, save_json

class ConfigurationManager:
    def __init__(
        self, 
        config_file_path=CONFIG_FILE_PATH,
        params_file_path=PARAM_FILE_PATH,
        schema_file_path=SCHEMA_FILE_PATH
        ):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)
        self.schema = read_yaml(schema_file_path)

        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self)->ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.LogisticRegression
        schema = self.schema.TARGET_COLUMNS 

        create_directories([config.root_dir])    

        model_evaluation_config = ModelEvaluationConfig(
            root_dir = config.root_dir,
            test_file_path = config.test_file_path,
            model_path = config.model_path,
            metric_file_name = config.metric_file_name,
            all_params = params,
            target_column = schema.name,
            mlflow_uri = config.mlflow_uri

        )            

        return model_evaluation_config



In [7]:
import os
import pandas as pd
from sklearn.metrics import accuracy_score, recall_score,precision_score,f1_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib

In [8]:
class ModelEvaluation:
    def __init__(self, config:ModelEvaluationConfig):
        self.config=config

    def eval_metrices(self, actual, pred):
        acc = accuracy_score(actual, pred)
        recall = recall_score(actual, pred)
        precision = precision_score(actual, pred)
        f1_scores = f1_score(actual, pred)

        return acc, recall, precision, f1_scores

    def log_into_mlflow(self):

        test_data = pd.read_csv(self.config.test_file_path)
        model = joblib.load(self.config.model_path)

        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[[self.config.target_column]]

        mlflow.set_tracking_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        # Enable autoLogging
        #mlflow.autolog()

        with mlflow.start_run():

            predicted_qualities = model.predict(test_x)
            (acc, recall, precision, f1_score) = self.eval_metrices(test_y, predicted_qualities)

            ## save metrices as local
            scores = {"accuracy": acc, "recall":recall, "precision":precision, "f1_score":f1_score}
            save_json(path=Path(self.config.metric_file_name), data=scores)

            mlflow.log_params(self.config.all_params)

            mlflow.log_metric("accuacr",acc)
            mlflow.log_metric("recall", recall)
            mlflow.log_metric("precision", precision)
            mlflow.log_metric("f1_score", f1_score)


            # local_model_path = Path("c:/Users/ojha.manish.kumar/Documents/Udemy/DataScience_End_to_End_Projects/mlflow_artifacts").resolve()
            # mlflow.sklearn.save_model(sk_model=model, path=str(local_model_path))

            # mlflow.log_artifacts(str(local_model_path), artifact_path="model")
            mlflow.sklearn.log_model(model, artifact_path="model")


In [9]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation = ModelEvaluation(config=model_evaluation_config)
    model_evaluation.log_into_mlflow()
except Exception as e:
     raise e

[2025-09-21 10:46:33,088: INFO: common]: yaml file: config\config.yaml loaded successfully
[2025-09-21 10:46:33,131: INFO: common]: yaml file: params.yaml loaded successfully
[2025-09-21 10:46:33,181: INFO: common]: yaml file: schema.yaml loaded successfully
[2025-09-21 10:46:33,190: INFO: common]: created directory at: artifacts
[2025-09-21 10:46:33,197: INFO: common]: created directory at: artifacts/model_evaluation
[2025-09-21 10:46:42,479: INFO: common]: json file saved at: artifacts\model_evaluation\metric.json
